# Mixing Station Workspace

***

Update the .csv file to include the desired constituent electrolytes and their volume ratios, making sure they match up to the correct numbered location on the physical workspace. The columns in the .csv file correspond to the following.. 

| Name | Volume (uL) | Starting Volume (mL) | Density (g/mL) | Aspirate Constant (mbar/uL) | Aspirate Speed (uL/s) |
| --- | --- | --- | --- | --- | --- |

Run the code block below and confirm that your total volume is correct.

*Note: When debugging, you can set `Active=false` in the [Device JSON](data/devices/mixing_stations.json), to execute the code without attempting to connect to the gantry or pipette systems.*

In [ ]:
from src.robot_controller import experiment_setup

experiment = experiment_setup.experiment(device_name="microtron_01")
experiment.read_csv(CSV_PATH="data/CSVs/electrolyte_recipe.csv")

***

## Tune Aspiration Variables

Based on the literature, we can expect the aspirate constant to be roughly equal to the system pressure $P_r$ divided by the reservoir volume $V_r$. The system pressure can be assumed equal to atmospheric pressure $\approx$ 1000mbar and the chosen reservoir volume is 2000uL $\implies$ the constant should be about **0.5mbar/uL**.

$$ \Delta P_r = \frac{P_s}{V_r} V_{asp} $$ 

For more viscous liquids, this value may increase based on the ohm's law equivalent of fluid flow through a pipe (the Hagen-Poiseuille equation). To compensate for this, an extra variable is included to slow down the rate of aspiration in an attempt to lower the pressure change required to aspirate more viscous fluids. Along with the response time of the pressure PID controller, the aspirate speed will determine how quickly the pressure of the disc pump changes. A typical value would be **10uL/s**.

$$ \Delta P = \frac{8 \mu L}{\pi R^4} Q $$ 

In [ ]:
%matplotlib inline
experiment.tune(name="example", pot_number=1, aspirate_volume=10, starting_volume=50, asp_const_range=[0.4, 0.6], asp_speed_range=[8, 10], N=5)

### Update this [CSV file](data/CSVs/electrolyte_recipe.csv) with the chosen variables before beginning an experiment! 

***

## Run Experiment

Running the below code block will begin a simple experiment that creates an electrolye mixture based on the volumes of constitutent electrolytes given in the CSV file.

In [ ]:
experiment.run(N=1)

***

## Future Work

Once the viability of the Mixing Station has been validated, we can look to use an API to call `experiment.run()` via the [Atinary software](https://scientia.atinary.com/sdlabs/academic/dashboard), enabling a self-driving lab setup where the constituent volumes are updated with each run.